In [1]:
# import the necessary packages
from imutils.video import VideoStream, FileVideoStream
from imutils import face_utils
import datetime
import argparse
import imutils
import time
import dlib
import cv2
from collections import OrderedDict
import numpy as np
import sys
from pose_estimator import PoseEstimator

In [2]:
FACIAL_LANDMARKS_IDXS = OrderedDict([
    ("nose", (27, 36)),
    ("face", (0, 26))
])

In [3]:
def visualize_facial_landmarks(image, shape, colors=None, alpha=0.75):
    # create two copies of the input image -- one for the
    # overlay and one for the final output image
    overlay = image.copy()
    output = image.copy()

    # if the colors list is None, initialize it with a unique
    # color for each facial landmark region
    if colors is None:
        colors = [(255, 255, 255), (0, 0, 0), (0, 0, 0), (0, 0, 0),
            (0, 0, 0), (0, 0, 0),
            (0, 0, 0), (0, 0, 0), (255, 255, 255)]
        colors = [(19, 199, 109), (79, 76, 240), (230, 159, 23),
            (168, 100, 168), (158, 163, 32),
            (163, 38, 32), (180, 42, 220), (100, 150, 250)]

    # loop over the facial landmark regions individually
    for (i, name) in enumerate(FACIAL_LANDMARKS_IDXS.keys()):
        # grab the (x, y)-coordinates associated with the
        # face landmark
        (j, k) = FACIAL_LANDMARKS_IDXS[name]
        pts = shape[j:k]

        # check if are supposed to draw the jawline
        if name == "jaw":
            # since the jawline is a non-enclosed facial region,
            # just draw lines between the (x, y)-coordinates
            for l in range(1, len(pts)):
                ptA = tuple(pts[l - 1])
                ptB = tuple(pts[l])
                cv2.line(overlay, ptA, ptB, colors[i], 2)

        # otherwise, compute the convex hull of the facial
        # landmark coordinates points and display it
        else:
            if name == 'face':
                hull = cv2.convexHull(pts)
                cv2.drawContours(overlay, [hull], -1, colors[i], -1)

    # apply the transparent overlay
    cv2.addWeighted(overlay, alpha, output, 1 - alpha, 0, output)

    # return the output image
    return output

# def compute_features(face_points):
#     assert (len(face_points) == 68), "len(face_points) must be 68"
    
#     face_points = np.array(face_points)
#     features = []
#     for i in range(68):
#         for j in range(i+1, 68):
#             features.append(np.linalg.norm(face_points[i]-face_points[j]))
            
#     return np.array(features).reshape(1, -1)

In [4]:
# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor_path = "../detect-face-parts/shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor(predictor_path)

# # initialize rpy model
# model = Sequential()
# model.add(Dense(units=20, activation='relu', kernel_regularizer='l2', input_dim=68*67))
# model.add(Dense(units=10, activation='relu', kernel_regularizer='l2'))
# model.add(Dense(units=3, activation='linear'))
# model = load_model('models/model.h5')
# std = StandardScaler()

filename = '../data/cohface/1/0/data.avi'
# filename = None
if filename:
    vs = FileVideoStream(filename).start()
else:
    vs = VideoStream(src=0).start()

In [5]:
# initialize the video stream and allow the cammera sensor to warmup
print("[INFO] camera sensor warming up...")
time.sleep(2.0)

# loop over the frames from the video stream
first_pass = True
while True:
    # grab the frame from the threaded video stream, resize it to
    # have a maximum width of 400 pixels, and convert it to
    # grayscale
    frame = vs.read()
    frame = imutils.resize(frame, width=400)
    if first_pass:
        height, width = frame.shape[:2]
        pose_estimator = PoseEstimator(img_size=(height, width))
        first_pass = False
        
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # detect faces in the grayscale frame
    rects = detector(gray)

    # loop over the face detections
    for rect in rects:
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        shape = face_utils.shape_to_np(predictor(gray, rect))
        
#         # determine roll pitch and yaw
#         rpy_features = std.transform(compute_features(shape))
#         roll, pitch, yaw = model.predict(rpy_features)[0]
#         print(' Roll: {:.2f}°'.format(roll))
#         print('Pitch: {:.2f}°'.format(pitch))
#         print('  Yaw: {:.2f}°'.format(yaw))

        # Try pose estimation with 68 points.
        pose = pose_estimator.solve_pose_by_68_points(shape.astype('float'))
        
#         # Stabilize the pose.
#         steady_pose = []
#         pose_np = np.array(pose).flatten()
#         for value, ps_stb in zip(pose_np, pose_stabilizers):
#             ps_stb.update([value])
#             steady_pose.append(ps_stb.state[0])
#         steady_pose = np.reshape(steady_pose, (-1, 3))


        # loop over the (x, y)-coordinates for the facial landmarks
        # and draw them on the image
        for (x, y) in shape:
            cv2.circle(frame, (x, y), 1, (0, 0, 255), -1)
            
        # loop over the face parts individually and draw them 
        for (name, (i, j)) in FACIAL_LANDMARKS_IDXS.items():
            # clone the original image so we can draw on it, then
            # display the name of the face part on the image
            clone = frame.copy()
            cv2.putText(clone, name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                0.7, (0, 0, 255), 2)

            # loop over the subset of facial landmarks, drawing the
            # specific face part
            for (x, y) in shape[i:j]:
                cv2.circle(clone, (x, y), 1, (0, 0, 255), -1)
                


            # extract the ROI of the face region as a separate image
#             (x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]]))
#             roi = frame[y:y + h, x:x + w]
#             roi = imutils.resize(roi, width=250, inter=cv2.INTER_CUBIC)
            

    # visualize all facial landmarks with a transparent overlay
    output = visualize_facial_landmarks(frame, shape)
    # Uncomment following line to draw pose annotation on frame.
    pose_estimator.draw_annotation_box(
            output, pose[0], pose[1], color=(255, 128, 128))
    # Uncomment following line to draw head axes on frame.
#     pose_estimator.draw_axes(frame, steady_pose[0], steady_pose[1])
    cv2.imshow("Frame", output)
#     # show the particular face part
#     cv2.imshow("ROI", roi)
#     cv2.imshow("ROI", clone) 
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

# do some cleanup
cv2.destroyAllWindows()
vs.stop()

[INFO] camera sensor warming up...
successful init


In [8]:
raw_value = []
with open('assets/model.txt') as file:
    for line in file:
        raw_value.append(line)
model_points = np.array(raw_value, dtype=np.float32)
model_points = np.reshape(model_points, (3, -1)).T

# Transform the model into a front view.
model_points[:, 2] *= -1

# Camera internals
focal_length = 400
camera_center = (400 / 2, 300 / 2)
camera_matrix = np.array(
    [[focal_length, 0, camera_center[0]],
     [0, focal_length, camera_center[1]],
     [0, 0, 1]], dtype="double")

# Assuming no lens distortion
dist_coeefs = np.zeros((4, 1))

# Rotation vector and translation vector
r_vec = np.array([[0.01891013], [0.08560084], [-3.14392813]])
t_vec = np.array([[-14.97821226], [-10.62040383], [-2053.03596872]])

def solve_pose_by_68_points(image_points):
    """
    Solve pose from all the 68 image points
    Return (rotation_vector, translation_vector) as pose.
    """

    (_, rotation_vector, translation_vector) = cv2.solvePnP(
        model_points,
        image_points,
        camera_matrix,
        dist_coeefs,
        rvec=r_vec,
        tvec=t_vec,
        useExtrinsicGuess=True)

    return (rotation_vector, translation_vector)
